# 3.5 출력층 설계하기
- 회귀: 보통 출력층의 뉴런 수가 한 개이므로 항등함수를 사용한다.
- 분류: 데이터가 어느 클래스에 속하는지 다루기 때문에 소프트맥스 함수를 사용한다.

소프트맥스 함수 구현

In [1]:
import numpy as np

In [2]:
a=np.array([0.3, 2.9, 4.0])
exp_a=np.exp(a)
print(exp_a)

[ 1.34985881 18.17414537 54.59815003]


In [3]:
sum_exp_a=np.sum(exp_a)
print(sum_exp_a)

74.1221542101633


In [4]:
y=exp_a/sum_exp_a
print(y)

[0.01821127 0.24519181 0.73659691]


In [5]:
# 사용자 정의 함수로 구현
def softmax(a):
    exp_a=np.exp(a)
    sum_exp_a=np.exp(exp_a)
    y=exp_a/sum_exp_a
    return y

소프트맥스 함수 구현 시 주의점<br>
exponential에 큰 값을 넣으면 오버플로우가 발생할 우려가 있다.

In [6]:
a=np.array([1010, 1000, 990])
np.exp(a)/np.sum(np.exp(a))

C:\Users\kikio\AppData\Local\Temp\ipykernel_3564\369000715.py:2: RuntimeWarning: overflow encountered in exp
  np.exp(a)/np.sum(np.exp(a))
C:\Users\kikio\AppData\Local\Temp\ipykernel_3564\369000715.py:2: RuntimeWarning: invalid value encountered in divide
  np.exp(a)/np.sum(np.exp(a))


array([nan, nan, nan])

In [7]:
c=np.max(a)
a-c

array([  0, -10, -20])

In [8]:
np.exp(a-c)/np.sum(np.exp(a-c)) # 입력값 중 최댓값을 입력에 빼줌으로써 지수함수가 커지는 것을 방지한다.

array([9.99954600e-01, 4.53978686e-05, 2.06106005e-09])

In [9]:
def softmax(a):
    c=np.max(a)
    exp_a=np.exp(a-c) # 오버플로우 대책
    sum_exp_a=np.sum(exp_a)
    y=exp_a/sum_exp_a
    return y

소프트맥스 함수의 출력 총합은 1이다. 이를 활용해 문제를 확률적으로 대응할 수도 있다.

In [10]:
a=np.array([0.3, 2.9, 4.0])
y=softmax(a)
print(y)

[0.01821127 0.24519181 0.73659691]


In [11]:
np.sum(y)

1.0

# 3.6 손글씨 숫자 인식

In [12]:
from mnist_original import load_mnist # 현재 디렉토리에 저장해서 사용
    
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False) 
# flatten은 입력 이미지를 1차원으로, normalize는 이미지 픽셀을 0~1값으로 정규화할지, one-hot-label은 원핫인코딩을 할지 결정한다.

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [13]:
import numpy as np
from PIL import Image

def img_show(img):
    pil_img=Image.fromarray(np.uint8(img))
    pil_img.show()

img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img=img.reshape(28,28) # 원래 이미지 모양으로 변형한다. 784 = 28 X 28
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


### 신경망의 추론 처리
- 입력층 뉴런 수: 28 x 28 = 784 개
- 출력층 뉴런 수: 숫자 0~9이므로 10 개

은닉층은 총 2개로 첫 번째 은닉층 뉴런 수는 50, 두 번째 은닉층 뉴런 수는 100 (임의로 설정한 값이다)

In [14]:
def get_data():
    (x_train, t_train), (x_test, t_test)= load_mnist(normalize =True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f: # 학습된 가중치가 저장되어 있는 pickle 파일을 불러온다.
        network = pickle.load(f)
        return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1)+b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2)+b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3)+b3
    y=softmax(a3)
    
    return y

In [22]:
x, t = get_data()
network = init_network()

accuracy_cnt=0
for i in range(len(x)):
    y = predict(network, x[i]) # x[i]에는 784개의 입력이 재재, y에는 10개의 확률에 대한 출력이 존재
    p = np.argmax(y) # 그 중에서 가장 높은 확률의 출력을 선택한다.
    if p==t[i]: # t_train값과 일치하면 count
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt)/len(x)))

FileNotFoundError: [Errno 2] No such file or directory: 'sample_weight.pkl'

### 배치 처리

In [16]:
#x, _ = get_data()
#network=init_network()
#w1, w2, w3 = network['w1'], network['w2'], network['w3']

#x.shape
#x[0].shape # (784, 0)
#w1.shape # (784, 50) 각 층의 뉴런 수와 일치
#w2.shape # (50, 100)
#w3.shape # (100, 10)

In [17]:
#x, t= get_data()
#network=init_network()

#batch_size=100
#accuracy_cnt=0

#for i in range(0, len(x), batch_size):
#    x_batch=x[i:i+batch_size]
#    y_natch=predict(network, x_batch)
#    p=np.argmax(y_batch, axis=1)
#    accuracy_cnt+=np.sum(p==t[i:i+batch_size])

#print("Accuracy:" + str(float(accuracy_cnt/len(x))

In [18]:
list(range(0, 10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [19]:
list(range(0,10,3))

[0, 3, 6, 9]

In [20]:
x=np.array([[0.1,0.8,0.1],[0.3,0.1,0.6],[0.2,0.5,0.3],[0.8,0.1,0.1]])
y=np.argmax(x,axis=1)
print(y)

[1 2 1 0]


In [21]:
y=np.array([1,2,1,0])
t=np.array([1,2,0,0])
print(y==t)
np.sum(y==t)

[ True  True False  True]


3